# Initial Steps

My first thoughts were to ensure that I can properly connect to the interactive brokers application. This was set-up under connector.ipynb and was very helpful for getting the target contract data. The code snippet below lets me connect. Grab hourly data for the last 2 years for the March 2024 contract, and then download it for later use.

In [ ]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10)

contract = Future('ES', '202403', 'CME')

ib.qualifyContracts(contract)

ib.reqMarketDataType(3)

historical_data = ib.reqHistoricalData(
    contract, endDateTime='', durationStr='10 Y',
    barSizeSetting='1 hour', whatToShow='TRADES', useRTH=True)

df = util.df(historical_data)

df.to_csv('es.csv')

ib.disconnect()

# Initial Data Analysis

I wanted to examine the data to see any trends, and do some initial simple modeling. I used the following code to do so.

In [3]:
import pandas as pd

path = "data/es.csv"

df= pd.read_csv(path, encoding="utf-8", index_col=0)

import plotly.graph_objects as go

df['date'] = pd.to_datetime(df['date'], utc=True)

# set to US/Central timezone
df['date'] = df['date'].dt.tz_convert('US/Central')

# Create a hover text string that includes date and prices for each point
hover_texts = df['date'].dt.strftime('%Y-%m-%d %H:%M') + '<br>Open: ' + df['open'].astype(str) + '<br>High: ' + df['high'].astype(str) + '<br>Low: ' + df['low'].astype(str) + '<br>Close: ' + df['close'].astype(str)

# Create an interactive candlestick chart using Plotly with custom hover text
fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['open'],
                                     high=df['high'],
                                     low=df['low'],
                                     close=df['close'],
                                     hovertext=hover_texts,
                                     hoverinfo='text')])  # Use 'text' for custom hovertext


# Find the index positions of the middle of each trading day
middle_indices = df.groupby(df['date'].dt.date).apply(lambda x: x.index[len(x)//2]).values

# Create custom x-axis labels based on the middle index positions
x_labels = df.loc[middle_indices, 'date'].dt.strftime('%m-%d')

# Correctly find the numeric index positions of the first and last entries for each trading day
first_indices = df.groupby(df['date'].dt.date).apply(lambda x: x.index[0]).values
last_indices = df.groupby(df['date'].dt.date).apply(lambda x: x.index[-1]).values


# Create vertical lines for the start and end of each trading day
shapes = [dict(type='line',
               x0=i, y0=df['low'].min(), x1=i, y1=df['high'].max(),
               xref='x', yref='y',
               line=dict(color='green', width=1, dash='dash'))
          for i in first_indices]

# Create a separate trace for the volume data
volume_trace = go.Bar(x=df.index, y=df['volume'], name='Volume', yaxis='y2')

# Add the volume trace to the figure
fig.add_trace(volume_trace)

# Customize the layout
fig.update_layout(
    title='Historical Prices',
    yaxis=dict(title='Price'),
    yaxis2=dict(title='Volume', overlaying='y', side='right'),
    xaxis=dict(
        tickmode='array',
        #tickvals=middle_indices,
        #ticktext=x_labels,
        tickangle=-45
    ),
    xaxis_rangeslider_visible=False,
    #shapes=shapes
)

# Display the interactive plot
fig.show()

In [4]:
# Line plot on the closing prices only
fig = go.Figure(data=go.Scatter(x=df.index, y=df['close'], mode='lines'))
fig.update_layout(title='Closing Prices', xaxis_title='Time', yaxis_title='Price')
fig.show()

# Thoughts on Plot

The price seems to be increasing steadily as of late, but there were some dips in the past that could be dangerous. For my first model, I am going to use linear regression to get a sense of the best fit hopefully capturing the trend.

In [6]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tulipy as ti

# Assuming your DataFrame is named 'df' and contains 'close', 'volume', 'high', and 'low' columns
df['index'] = df.index.astype(int)

# cos_index
df['cos_index'] = np.cos(df.index)
df['sin_index'] = np.sin(df.index)
df['prev_close'] = df['close'].shift(1)
df['prev_volume'] = df['volume'].shift(1)

# Do log volume with care, as it can be zero
df['log_volume'] = np.log(df['volume'] + 1)
df['prev_log_volume'] = df['log_volume'].shift(1)

# Drop any missing values from the DataFrame
data = df.dropna()

from sklearn.preprocessing import StandardScaler

# Prepare the data for linear regression
X = data[['index','prev_log_volume']]
# scale the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

X = sm.add_constant(X)
y = data['close']

# Fit the linear regression model
model = sm.OLS(y, X).fit()
print(model.summary())
# print explained variance
print(model.rsquared)
# Make predictions using the model
predictions = model.predict(X)

# Create a new DataFrame to store the predictions
pred_df = pd.DataFrame({'close': data['close'], 'predictions': predictions}, index=data.index)

# Create the line plot on the closing prices
fig = go.Figure(data=go.Scatter(x=df.index, y=df['close'], mode='lines', name='Closing Prices'))

# Add the predicted values to the plot
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['predictions'], mode='lines', name='Predicted Prices'))

# Customize the layout
fig.update_layout(title='Closing Prices with Linear Regression',
                  xaxis_title='Time',
                  yaxis_title='Price')

# Display the interactive plot
fig.show()

                            OLS Regression Results                            
Dep. Variable:                  close   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.588
Method:                 Least Squares   F-statistic:                     1188.
Date:                Wed, 13 Mar 2024   Prob (F-statistic):          3.00e-321
Time:                        23:11:16   Log-Likelihood:                -10894.
No. Observations:                1667   AIC:                         2.179e+04
Df Residuals:                    1664   BIC:                         2.181e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4601.9274      4.086   1126.314      0.0

# Linear Regression Results

The model is not great, but it signals an upward trend with time (index in this case). Volume introduces some noise but does not fully capture the trend. For now, this has convinced me to buy an initial position in the contract and hold it for the remainder of the competition. I will definitely need more data, and a better model to create a more confident strategy.

![order 1](images/order_1.png)



# News Data

My first step after this is to collect news data, and align it with the data I have. I found the following github repository:

url: https://github.com/Zdong104/FNSPID_Financial_News_Dataset/tree/main

That provides code for scraping financial news data from the web. I will use this to collect news data for the past couple of years and work with that.

# Target Stocks

The dataset above is pretty intensive going back as far as 1999. The market dynamics probably changed since then, and the SP500 list probably did as well, so I will only use the current SP500 list. One interesting thing to note is that the SP500 list is weighted, which could be a useful detail to include in my model. For now I am going to scrape the list of stocks and their weights.

![weights_example](images/weights.png)

The code below was used to scrape the current SP500 list and their weights for more recent news (Last 3 months or so). For older news I downloaded the entire dataset from HuggingFace, and cleaned it up to only include the last 2 years of data, and only the current SP500 list.

url: https://huggingface.co/datasets/Zihan1004/FNSPID/tree/main/Stock_news

# Notes on Dataset

The dataset was pretty massive (20GB) but it extended all the way back to 1999. I used the following code to clean it up and only include the last 2 years of data, and only the current SP500 list.

# Scrape Code

In [ ]:
import random
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/93.0.961.47 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
]

def get_webdriver():
    random_user_agent = random.choice(user_agents)
    options = Options()
    options.add_argument(f"user-agent={random_user_agent}")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--log-level=3')
    prefs = {"profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)
    return driver

def get_sp500_stocks(driver):
    url = "https://www.slickcharts.com/sp500"
    driver.get(url)
    
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "table-borderless"))
    )
    
    stock_data = []
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    for row in rows[1:]:  # Skip the header row
        cells = row.find_elements(By.TAG_NAME, "td")
        symbol = cells[2].text
        company = cells[1].text
        weight = cells[3].text.rstrip("%")
        
        stock_data.append({
            "Symbol": symbol,
            "Company": company,
            "Weight": weight
        })
    
    return pd.DataFrame(stock_data)


driver = get_webdriver()

try:
    sp500_stocks = get_sp500_stocks(driver)
    sp500_stocks["Weight"] = sp500_stocks["Weight"].astype(float) / 100
    sp500_stocks.to_csv("sp500_stocks.csv", index=False)
    print("S&P 500 stocks and weights scraped successfully!")
except Exception as e:
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

# Clean-up code

In [ ]:
import pandas as pd

data_path = ""
stock_list_path = ""
stock_list = pd.read_csv(stock_list_path)
stock_list = stock_list['Stock_name'].tolist()

from tqdm import tqdm

# Get the total number of rows in the CSV file
total_rows = sum(1 for _ in open(data_path, 'r', encoding='utf-8'))

# Calculate the total number of chunks
chunksize = 1000
total_chunks = total_rows // chunksize

# Read the data from the file in chunks, treating 'Unnamed: 0' as the index column
chunk_iter = pd.read_csv(data_path, chunksize=chunksize)

# Initialize an empty list to collect filtered DataFrame chunks
filtered_chunks = []

# Initialize variables for rolling average calculation
total_rows_processed = 0
total_rows_kept = 0

# Iterate over each DataFrame chunk with progress bar
with tqdm(total=total_chunks, desc="Processing chunks", unit="chunk") as pbar:
    for chunk in chunk_iter:
        # Convert the 'Date' column to datetime type
        chunk['Date'] = pd.to_datetime(chunk['Date'])    
    
        # Filter the chunk to include only articles dated 2022/12/27 and higher
        filtered_chunk = chunk[chunk['Date'] >= '2022-12-27']
        
        # Filter the chunk to include only the stocks in the stock list
        filtered_chunk = filtered_chunk[filtered_chunk['Stock_symbol'].isin(stock_list)]
        
        # Append the filtered chunk to the list
        filtered_chunks.append(filtered_chunk)
        
        # Update rolling average calculation
        total_rows_processed += len(chunk)
        total_rows_kept += len(filtered_chunk)
        
        # Calculate the rolling average percentage
        rolling_avg_percentage = (total_rows_kept / total_rows_processed) * 100
        
        # Update the progress bar description with the rolling average percentage
        pbar.set_description(f"Processing chunks (Rolling Avg: {rolling_avg_percentage:.2f}%)")
        pbar.update(1)

# Concatenate all the filtered chunks into a single DataFrame
filtered_data = pd.concat(filtered_chunks)

filtered_data.to_csv(data_path, index=False)

# Scraping more data

It turns out the first dataset only had articles up to 01/09/2024. They had provided code on their github on how to scrape more data, so I used that to get articles that were up to date up till around 03/14/2024. Then I combined all the data into one dataset formatted in the following way:

date - symbol - text

# Sentiment Analysis Additional Feature

Now that I have the news data, I want to calculate scores for each article, and then aggregate them by day, so that it acts as an overall sentiment for the day, and could maybe be used as a feature in my model. I used the following code to do so.

In [ ]:
import pandas as pd
import numpy as np
import torch

df = pd.read_csv(data_path)

# Use a pipeline as a high-level helper
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

from tqdm import tqdm
import ast
from torch.utils.data import DataLoader

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to GPU if available
model.to(device)

def analyze_sentiment(text_batch, chunk_size=512, stride=256):
    inputs = tokenizer(text_batch, return_tensors="pt", padding=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    sentiment_scores = []
    token_count = input_ids.size(1)

    if token_count <= chunk_size:
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)
        sentiment_scores.append(probs.cpu().numpy())
    else:
        for i in range(0, token_count, stride):
            chunk_input_ids = input_ids[:, i:i+chunk_size]
            chunk_attention_mask = attention_mask[:, i:i+chunk_size]

            with torch.no_grad():
                outputs = model(chunk_input_ids, attention_mask=chunk_attention_mask)
                logits = outputs.logits
                probs = torch.softmax(logits, dim=1)

            sentiment_scores.append(probs.cpu().numpy())

    sentiment_scores = np.mean(sentiment_scores, axis=0)
    return sentiment_scores[:, 0], sentiment_scores[:, 1], sentiment_scores[:, 2]

# Create a DataLoader for batching the input data
batch_size = 32
text_dataset = df['Text'].tolist()
dataloader = DataLoader(text_dataset, batch_size=batch_size)

# Wrap the model with DataParallel for multi-GPU support
model = torch.nn.DataParallel(model)

# Analyze sentiment in batches using tqdm for progress tracking
negative_scores = []
neutral_scores = []
positive_scores = []

for batch in tqdm(dataloader, desc="Analyzing Sentiment"):
    negative, neutral, positive = analyze_sentiment(batch)
    negative_scores.extend(negative)
    neutral_scores.extend(neutral)
    positive_scores.extend(positive)

df['negative'] = negative_scores
df['neutral'] = neutral_scores
df['positive'] = positive_scores

df.to_csv(data_path, index=False)

# Performance

![Sentiment](./images/lin_reg_sentiment.JPG)

Slightly better, but still not fully explaining the trend of the prices. I am thinking adding more features to try to capture a better image of the state of the market. In my mind I am thinking of adding features like interest rates, inflation, and other macroeconomic indicators. FRED has a lot of these indicators, so I will use that to get the data. The following code is used to get data from FRED.


In [ ]:
from fredapi import Fred
fred = Fred(api_key=api_key)

# Add series as a column to the dataframe match based on date
def add_series_to_df(df, series_id):
    series = fred.get_series(series_id, observation_start=min_date, observation_end=max_date)
    series_df = pd.DataFrame(series, columns=[series_id])
    series_df['date'] = series_df.index
    df = df.merge(series_df, on='date', how='left')
    return df

# Update
After adding what I had for the sentiment data, I realized that I unfortunately did not have enough data for meaningful training. I had a few hundred data points (12/2022 - 03/2024). I am not able to scrape news data for too far in the past, so I decided to just opt for more historical price data, without the sentiment part.